In [ ]:
!pip install -q kaggle

In [ ]:
from  google.colab import files
files.upload()

Saving kaggle (2).json to kaggle (2) (1).json


{'kaggle (2) (1).json': b'{"username":"anudeepdivi","key":"332e87256fa9dda71fe5572e44669633"}'}

In [ ]:
!kaggle datasets download -d samlearner/letterboxd-movie-ratings-data

Dataset URL: https://www.kaggle.com/datasets/samlearner/letterboxd-movie-ratings-data
License(s): Attribution 4.0 International (CC BY 4.0)
letterboxd-movie-ratings-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from zipfile import ZipFile
dataset = "/content/letterboxd-movie-ratings-data.zip"
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('dataset extracted')

dataset extracted


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
ratings = pd.read_csv('ratings_export.csv')
ratings['user_id'] = ratings['user_id'].astype(str)
movie_data = pd.read_csv('movie_data.csv', lineterminator='\n')
movie_data.head()


,_id,genres,image_url,imdb_id,imdb_link,movie_id,movie_title,original_language,overview,popularity,production_countries,release_date,runtime,spoken_languages,tmdb_id,tmdb_link,vote_average,vote_count,year_released
0,5fc85f606758f69634496fd3,"[""Music"",""Animation""]",film-poster/4/6/4/4/4/0/464440-football-freaks...,NaN,NaN,football-freaks,Football Freaks,en,"Football crazy, football mad. Don’t watch this...",0.600,"[""United Kingdom""]",1971-12-05,0.0,[],535272.0,https://www.themoviedb.org/movie/535272/,0.0,0.0,1971.0
1,5fc85ff26758f696344ace0c,[],film-poster/2/4/5/5/0/0/245500-aftermath-0-230...,tt0586129,http://www.imdb.com/title/tt0586129/maindetails,aftermath-1960,Aftermath,en,Aftermath was the pilot for an unsold TV serie...,0.600,[],1960-04-17,22.0,[],318331.0,https://www.themoviedb.org/movie/318331/,8.0,1.0,1960.0
2,5fc85f606758f69634496fcd,"[""Drama""]",film-poster/9/3/3/1/8/93318-where-chimneys-are...,tt0045731,http://www.imdb.com/title/tt0045731/maindetails,where-chimneys-are-seen,Where Chimneys Are Seen,ja,Gosho’s most celebrated film both in Japan and...,1.568,"[""Japan""]",1953-03-05,108.0,"[""日本語""]",117779.0,https://www.themoviedb.org/movie/117779/,6.6,10.0,1953.0
3,5fc85f606758f69634496fd1,"[""Drama""]",NaN,tt0187327,http://www.imdb.com/title/tt0187327/maindetails,the-musicians-daughter,The Musician's Daughter,en,Carl Wagner's good wife was dying. His heart b...,0.600,"[""United States of America""]",1911-12-12,15.0,[],560377.0,https://www.themoviedb.org/movie/560377/,0.0,0.0,1911.0
4,5fc85f606758f69634496fd4,"[""Documentary""]",film-poster/4/5/4/6/0/3/454603-50-years-of-fab...,tt4769914,http://www.imdb.com/title/tt4769914/maindetails,50-years-of-fabulous,50 Years of Fabulous,en,50 Years of Fabulous recounts the rich history...,0.600,[],2018-05-17,75.0,[],525187.0,https://www.themoviedb.org/movie/525187/,0.0,0.0,2018.0


In [ ]:
from scipy.sparse import csr_matrix

def create_user_item_matrix(ratings, min_user_ratings=1, min_movie_ratings=5):

    # Filter out users with fewer than min_user_ratings ratings
    user_counts = ratings['user_id'].value_counts()
    active_users = user_counts[user_counts >= min_user_ratings].index

    ratings_filtered = ratings[ratings['user_id'].isin(active_users)]

    # Filter out movies with fewer than min_movie_ratings ratings
    movie_counts = ratings_filtered['movie_id'].value_counts()
    active_movies = movie_counts[movie_counts >= min_movie_ratings].index

    ratings_filtered = ratings_filtered[ratings_filtered['movie_id'].isin(active_movies)]

    # Now create the user-item matrix with the filtered data
    user_ids = ratings_filtered['user_id'].unique()
    movie_ids = ratings_filtered['movie_id'].unique()

    user_id_to_index = {user_id: index for index, user_id in enumerate(user_ids)}
    movie_id_to_index = {movie_id: index for index, movie_id in enumerate(movie_ids)}

    row_indices = ratings_filtered['user_id'].map(user_id_to_index)
    col_indices = ratings_filtered['movie_id'].map(movie_id_to_index)
    data = ratings_filtered['rating_val']

    user_item_matrix = csr_matrix((data, (row_indices, col_indices)), shape=(len(user_ids), len(movie_ids)))

    return user_item_matrix

# Create the reduced user-item matrix
user_item_matrix = create_user_item_matrix(ratings)

In [ ]:
# Get the shape of the matrix (rows, columns)
rows, cols = user_item_matrix.shape

# Print the dimensions
print(f"Total rows: {rows}")
print(f"Total columns: {cols}")

Total rows: 7477
Total columns: 106188


In [ ]:
# Step 3: CF with SVD
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=50, random_state=42)
user_factors = svd.fit_transform(user_item_matrix)
item_factors = svd.components_
predicted_ratings_cf = np.dot(user_factors, item_factors)

In [ ]:
# Preprocess movie genres for Diversity (ILD)
movie_data['genres'] = movie_data['genres'].apply(lambda x: x.split('|') if isinstance(x, str) else [])
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(movie_data['genres'])


In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
def calculate_mae(user_item_matrix_target, predicted_ratings_cf):
    """
    Calculate Mean Absolute Error (MAE).
    """
    # Ensure both are dense NumPy arrays for consistent masking
    if hasattr(user_item_matrix_target, 'toarray'):
        user_item_matrix_target = user_item_matrix_target.toarray()

    # Create the mask based on the target user's ratings
    mask = user_item_matrix_target[0] > 0  # Extract the target user's ratings and create the mask

    # Apply the mask to both actual and predicted ratings, ensuring shape consistency
    actual = user_item_matrix_target[0][mask]  # Target user's actual ratings using the mask
    predicted = predicted_ratings_cf[0, mask]  # Corresponding predicted ratings using the same mask

    return mean_absolute_error(actual, predicted) if len(actual) > 0 else float('inf')

In [ ]:
def calculate_ild(recommendations, genres_encoded):
    """
    Calculate Intra-List Diversity (ILD).
    """
    if len(recommendations) < 2:
        return 0  # Diversity is zero if there is only one or no recommendations
    genre_vectors = genres_encoded[recommendations]
    pairwise_sim = cosine_similarity(genre_vectors)
    dissimilarity = 1 - pairwise_sim
    num_pairs = len(recommendations) * (len(recommendations) - 1) / 2
    ild = np.sum(dissimilarity[np.triu_indices_from(dissimilarity, k=1)]) / num_pairs
    return ild

In [ ]:
def calculate_novelty(recommendations, movie_data):
    """
    Calculate Novelty using Self-Information (based on movie popularity).
    """
    popularity = movie_data.iloc[recommendations]['popularity'].values
    return -np.sum(np.log2(popularity + 1e-9)) / len(recommendations) if len(recommendations) > 0 else 0


In [ ]:
def calculate_coverage(recommendations, movie_data):
    """
    Calculate Coverage: Fraction of movies covered in recommendations.
    """
    unique_movies = np.unique(recommendations)
    return len(unique_movies) / movie_data.shape[0]

In [ ]:
def evaluate_fitness(individual, user_item_matrix_target, predicted_ratings_cf, movie_data, genres_encoded):
    """
    Modified Fitness Function to ensure non-negative fitness scores
    """
    mae = calculate_mae(user_item_matrix_target, predicted_ratings_cf)
    # Shift accuracy to be non-negative, you can adjust the shift value (10 in this case)
    accuracy = 10 - mae
    ild = calculate_ild(individual, genres_encoded)
    novelty = calculate_novelty(individual, movie_data)
    coverage = calculate_coverage(individual, movie_data)

    # Ensure all components are non-negative
    accuracy = max(0, accuracy)
    ild = max(0, ild)
    novelty = max(0, novelty)
    coverage = max(0, coverage)

    return 0.4 * accuracy + 0.3 * ild + 0.2 * novelty + 0.1 * coverage

In [ ]:
import pandas as pd

def genetic_algorithm_for_user(
    target_user_id,  # Pass the string user_id here
    user_item_matrix,
    predicted_ratings_cf,
    movie_data,
    genres_encoded,
    user_id_mapping,  # Dictionary mapping user_ids to indices
    fitness_function,
    Gq=20,
    Pop_size=35,
    M=4,
    D=10,
    Cp=1.0,
    Mp=None
):
    # Set mutation probability
    if Mp is None:
        Mp = 1 / D

    # Map the string user_id to the corresponding matrix index
    if target_user_id not in user_id_mapping:
        print(f"Warning: User ID '{target_user_id}' not found in mapping. Returning None.")
        return None, None  # or handle appropriately

    user_index = user_id_mapping[target_user_id]

    # Get movies not rated by the target user
    unrated_movies = np.where(user_item_matrix[user_index].toarray().flatten() == 0)[0]

    # Initialize population with random recommendations
    population = [np.random.choice(unrated_movies, size=D, replace=False) for _ in range(Pop_size)]

    # Fitness Function Wrapping
    def calculate_fitness(individual):
        return fitness_function(
            individual,
            user_item_matrix[user_index],
            predicted_ratings_cf,
            movie_data,
            genres_encoded
        )

    # Selection, Crossover, Mutation
    def select_parents(population, fitness_scores):
      fitness_scores = np.nan_to_num(fitness_scores, nan=1e-10)
      if np.sum(fitness_scores) == 0:
        idx = np.random.choice(len(population), size=2, replace=False)
      else:
        probabilities = fitness_scores / np.sum(fitness_scores)
        probabilities /= np.sum(probabilities)
        idx = np.random.choice(len(population), size=2, p=probabilities, replace=False)
        return population[idx[0]], population[idx[1]]

    def crossover(parent1, parent2):
        if np.random.rand() < Cp:
            point = np.random.randint(1, len(parent1))
            child1 = np.concatenate((parent1[:point], parent2[point:]))
            child2 = np.concatenate((parent2[:point], parent1[point:]))
            return child1, child2
        return parent1, parent2

    def mutate(individual):
        if np.random.rand() < Mp:
            idx = np.random.randint(0, len(individual))
            individual[idx] = np.random.choice(unrated_movies)
        return individual

    # Genetic Algorithm Execution
    best_individual = None
    best_fitness = None

    for generation in range(Gq):
        # Evaluate fitness for each individual
        fitness_scores = np.array([calculate_fitness(ind) for ind in population])

        # Select the best individual in the current generation
        best_idx = np.argmax(fitness_scores)
        if best_fitness is None or fitness_scores[best_idx] > best_fitness:
            best_fitness = fitness_scores[best_idx]
            best_individual = population[best_idx]

        # Create next generation
        new_population = []
        for _ in range(Pop_size // 2):
            parent1, parent2 = select_parents(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([mutate(child1), mutate(child2)])

        population = new_population

    return best_individual, best_fitness


In [ ]:
user_exports = pd.read_csv('/content/users_export.csv')

# Create a mapping of user_id (string) to matrix index
user_ids = ratings['user_id'].unique()  # Get user IDs directly from ratings
user_id_mapping = {user_id: index for index, user_id in enumerate(user_ids)}


In [ ]:
best_recommendations, best_score = genetic_algorithm_for_user(
    target_user_id='richmond_hill',
    user_item_matrix=user_item_matrix,
    predicted_ratings_cf=predicted_ratings_cf,
    movie_data=movie_data,
    genres_encoded=genres_encoded,
    user_id_mapping=user_id_mapping,
    fitness_function=evaluate_fitness
)
def get_movie_names(recommendations, movie_data):
    """
    Retrieve movie names for the recommended indices.
    """
    recommended_movie_ids = movie_data.iloc[recommendations]['movie_id'].values
    recommended_movie_names = movie_data.iloc[recommendations]['movie_title'].values
    # Use the built-in zip function explicitly:
    return list(__builtins__.zip(recommended_movie_ids, recommended_movie_names))

# Display movie names for the best recommendations
recommended_movies = get_movie_names(best_recommendations, movie_data)

print("Recommended Movies for Target User:")
for movie_id, movie_name in recommended_movies:
    print(f"Movie ID: {movie_id}, Movie Name: {movie_name}")



Recommended Movies for Target User:
Movie ID: allsangsscenen-ar-din-hakan-hellstrom, Movie Name: Allsångsscenen är din Håkan Hellström
Movie ID: salmon-men, Movie Name: Salmon Men
Movie ID: ss-coptic-at-dock, Movie Name: S.S. 'Coptic' at Dock
Movie ID: agent-vinod-1977, Movie Name: Agent Vinod
Movie ID: nasser, Movie Name: Nasser
Movie ID: half-cocked, Movie Name: Half-Cocked
Movie ID: jaoon-kahan-bata-ae-dil, Movie Name: Lovefucked
Movie ID: for-socks-sake, Movie Name: For Sock's Sake
Movie ID: foyles-war-war-games, Movie Name: Foyle's War - War Games
Movie ID: animal-indireto, Movie Name: Animal Indireto
